In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import json

import polygon 
import gpx
import sql
import model 

In [2]:
cp_filepath = "../data/geojson/the_bridle.json"
run_filepath = "../data/stravaGPX/AnotherBridleContribution.gpx"
run2_filepath = "../data/stravaGPX/MorningOnTheRes.gpx"

In [3]:
def get_geojson(filepath):
    file = open(filepath, "r") 
    return json.load(file)

def get_coordinates_json(filepath):
    data = get_geojson(filepath)
    return data

def get_coordinates_list(filepath, p=True):
    data = get_coordinates_json(filepath)['features'][0]['geometry']['coordinates']
    if p:
        return data[0]
    return data

In [4]:
data = get_coordinates_list(cp_filepath)

In [ ]:
f = gpx.coordinates_dataframe_from_gpx(run2_filepath)

In [ ]:
df = f.assign(inside=lambda d: d.apply(polygon.row_inside_polygon, args=[data], axis=1))

In [ ]:
totals = polygon.twoD_total_inside_polygon(df)

In [ ]:
d = (2019, 9, 21)
distance = int(totals['distance'])
time = totals['time_delta'].seconds
model.add_run_from_athlete('eileen', d, distance, time)

In [5]:
from stravalib.client import Client

In [6]:

TYPES = ['time', 'latlng', 'altitude', 'heartrate', 'temp', ]

In [7]:
client = Client(access_token=TOKEN)
activities = client.get_activities(limit=10)

In [8]:
a = list(activities)

In [9]:
i = a[0].id

In [54]:
import datetime
def calculate_epoch(y, m, d):
    return int(datetime.datetime(y, m, d, 0, 0, tzinfo=datetime.timezone.utc).timestamp())

def get_strava_date(date):
    "There is a nicer way to do this but this is fine for now"
    return calculate_epoch(*[int(d) for d in date.split("T")[0].split("-")])

In [25]:
def get_strava_gpx_dataframe(activity_id, client, types=['time', 'latlng', 'altitude']):
    streams = client.get_activity_streams(activity_id, resolution='medium', types=types)
    latlng = pd.DataFrame(streams['latlng'].data, columns=['latitude', 'longitude'])
    time = pd.DataFrame(streams['time'].data, columns=['time'])
    if 'altitude' in streams:
        altitude = pd.DataFrame(streams['altitude'].data, columns=['elevation'])
        return pd.concat([latlng, altitude, time], axis=1)
    return pd.concat([latlng, altitude, time])

In [29]:
def strava_run_in_polygon(activity_id, client, types):
    return (
        get_strava_gpx_dataframe(activity_id, client, types)
        .assign(inside=lambda d: d.apply(polygon.row_inside_polygon, args=[data], axis=1))
    )

In [30]:
data = get_coordinates_list(cp_filepath)
strava = strava_run_in_polygon(i, client, TYPES)

In [55]:
s = polygon.totals_inside_polygon(strava)

In [67]:
datetime.datetime.strptime("2019_12_31", '%Y_%m_%d')

datetime.datetime(2019, 12, 31, 0, 0)